# Проект 3. О вкусной и здоровой пище

Первоначальная версия датасета состоит из десяти столбцов, содержащих следующую информацию:

* Restaurant_id — идентификационный номер ресторана / сети ресторанов;
* City — город, в котором находится ресторан;
* Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
* Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
* Rating — рейтинг ресторана по данным TripAdvisor (целевая переменная (!));
* Price Range — диапазон цен в ресторане;
* Number of Reviews — количество отзывов о ресторане;
* Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
* URL_TA — URL страницы ресторана на TripAdvisor;
* ID_TA — идентификатор ресторана в базе данных TripAdvisor.

# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import collections
import re
import requests
from patsy.splines import bs

In [42]:
df = pd.read_csv('main_task.csv')

In [45]:
def split_string(string):
    return string.strip('[]').replace("'", '').split(', ')

def map_cuisines(values, cuisines):
    return [1 if values is not np.nan and key[8:] in split_string(values) else 0 for key, val in cuisines.items()]

In [46]:
df = df.rename(columns={'Restaurant_id_x': 'Restaurant_id'})

In [47]:
df.info()

# переведём в нижний регистр названия столбцов
df.columns = df.columns.str.lower()
# заменим пробелы в названиях столбцов на символ нижний подчерк
df.columns = df.columns.str.replace(' ', '_')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [48]:
df.head()

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


### Restaurant_id

In [49]:
df.restaurant_id.value_counts()

id_436      18
id_633      18
id_227      18
id_871      18
id_585      17
            ..
id_13055     1
id_7398      1
id_13775     1
id_5406      1
id_6421      1
Name: restaurant_id, Length: 11909, dtype: int64

In [50]:
# Заодно проверяем другие значения которые по идею должны быть уникальными
# Усли найдутся дупликаты, удаляем
df.id_ta.value_counts()

d7342803     2
d13002276    2
d1315077     2
d7809594     2
d4600226     2
            ..
d12640107    1
d12968785    1
d1854949     1
d5481459     1
d8114710     1
Name: id_ta, Length: 39980, dtype: int64

In [51]:
df[df.id_ta == 'd11999956']

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta
16920,id_3536,Madrid,"['International', 'Mediterranean', 'Spanish', ...",3538.0,4.0,$$ - $$$,52.0,"[['Great Food! Wide Variety.', 'Dinner'], ['11...",/Restaurant_Review-g187514-d11999956-Reviews-S...,d11999956
29879,id_3662,Madrid,"['International', 'Mediterranean', 'Spanish', ...",3664.0,4.0,$$ - $$$,52.0,"[['Great Food! Wide Variety.', 'Dinner'], ['11...",/Restaurant_Review-g187514-d11999956-Reviews-S...,d11999956


In [52]:
df.drop_duplicates(subset=['id_ta'], inplace=True)

In [53]:
df.url_ta.value_counts().values[:50]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1], dtype=int64)

In [54]:
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39980 entries, 0 to 39979
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      39980 non-null  object 
 1   city               39980 non-null  object 
 2   cuisine_style      30701 non-null  object 
 3   ranking            39980 non-null  float64
 4   rating             39980 non-null  float64
 5   price_range        26101 non-null  object 
 6   number_of_reviews  37437 non-null  float64
 7   reviews            39980 non-null  object 
 8   url_ta             39980 non-null  object 
 9   id_ta              39980 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


Идея! Можем группировать строки с помошью этой колонки и получить общую информацию о сети ресторанов.

### city

In [55]:
print('Число городов', len(df.city.value_counts()))
df.city.value_counts()

Число городов 31


London        5757
Paris         4897
Madrid        3088
Barcelona     2734
Berlin        2155
Milan         2133
Rome          2078
Prague        1443
Lisbon        1300
Vienna        1166
Amsterdam     1086
Brussels      1060
Hamburg        949
Munich         893
Lyon           892
Stockholm      820
Budapest       816
Warsaw         727
Dublin         673
Copenhagen     659
Athens         628
Edinburgh      596
Zurich         538
Oporto         513
Geneva         481
Krakow         443
Oslo           385
Helsinki       376
Bratislava     301
Luxembourg     210
Ljubljana      183
Name: city, dtype: int64

Идея! Похожее ситуация с предыдущим. Можем группировать по городам.


### cuisine_style

In [56]:
df.cuisine_style.value_counts()

['Italian']                                                                                     1032
['French']                                                                                       805
['Spanish']                                                                                      692
['French', 'European']                                                                           405
['Cafe']                                                                                         403
                                                                                                ... 
['Thai', 'Asian', 'Vegetarian Friendly', 'Vegan Options']                                          1
['Steakhouse', 'Argentinean', 'South American', 'Grill', 'Gluten Free Options']                    1
['Irish', 'Cafe', 'European', 'Vegan Options', 'Gluten Free Options', 'Vegetarian Friendly']       1
['Bar', 'Gastropub', 'Mediterranean', 'European', 'Spanish', 'Pub']                        

In [57]:
df['cuisines_count'] = df.cuisine_style.apply(lambda x: len(split_string(x)) if x is not np.nan else 1)
cuisine_styles = collections.Counter()

for row in df[df.cuisine_style.notna()].cuisine_style:
    for element in split_string(row):
        cuisine_styles[element] += 1

# Создаем массив новых категориальных колонок из стиоей кухонь будем учитывать только стили встречающиеся больше 500 раз
selected_cuisines = {'Cuisine ' + k: v  for k, v in sorted(cuisine_styles.items(), key=lambda item: item[1])}
selected_cuisines = dict(filter(lambda x: True if x[1] > 500 else False, selected_cuisines.items()))
selected_cuisines

{'Cuisine Vietnamese': 513,
 'Cuisine Contemporary': 523,
 'Cuisine Barbecue': 555,
 'Cuisine Steakhouse': 573,
 'Cuisine Fusion': 577,
 'Cuisine Czech': 595,
 'Cuisine Halal': 597,
 'Cuisine Greek': 604,
 'Cuisine Healthy': 620,
 'Cuisine German': 661,
 'Cuisine Wine Bar': 697,
 'Cuisine Thai': 743,
 'Cuisine Middle Eastern': 781,
 'Cuisine Indian': 1041,
 'Cuisine Portuguese': 1107,
 'Cuisine Chinese': 1145,
 'Cuisine Sushi': 1156,
 'Cuisine American': 1312,
 'Cuisine Central European': 1392,
 'Cuisine Japanese': 1463,
 'Cuisine Seafood': 1504,
 'Cuisine International': 1583,
 'Cuisine British': 1595,
 'Cuisine Fast Food': 1705,
 'Cuisine Cafe': 2325,
 'Cuisine Pub': 2449,
 'Cuisine Spanish': 2788,
 'Cuisine Pizza': 2849,
 'Cuisine Asian': 3010,
 'Cuisine French': 3189,
 'Cuisine Bar': 3296,
 'Cuisine Gluten Free Options': 4110,
 'Cuisine Vegan Options': 4482,
 'Cuisine Italian': 5963,
 'Cuisine Mediterranean': 6271,
 'Cuisine European': 10056,
 'Cuisine Vegetarian Friendly': 11183}

In [179]:
# # Создаем новый датафрейм из стилей кухонь
# cuisines = pd.DataFrame(columns=['id_ta'] + list(selected_cuisines.keys()))
# # заполняем датафрейм со значениями 1 если в строке встречается нужный стиль кухни
# for row in df[['id_ta', 'cuisine_style']].loc:
#     cuisines.loc[row.name] = [row['id_ta']] + map_cuisines(row['cuisine_style'], selected_cuisines)
# #

In [58]:
# Это займет много времени поэтому сохраним датафрейм в отдельный файл
# cuisines.to_csv('cuisines.csv')
cuisines = pd.read_csv('cuisines.csv', index_col=[0])

### price_range

In [59]:
df.price_range.value_counts()

$$ - $$$    18402
$            6276
$$$$         1423
Name: price_range, dtype: int64

In [60]:
df[df.city == 'London'].price_range.value_counts()

$$ - $$$    2908
$           1025
$$$$         214
Name: price_range, dtype: int64

In [61]:
# понятно что группировка по ресторанам может дать более точную информацию
# Если не сможем определить ценовой сегмент ресторана, ставим самый часто встречающийся сегмент.
grouped = df[['price_range', 'restaurant_id']].groupby(['restaurant_id'], as_index=False).agg(lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else '$$ - $$$')

In [62]:
# df['price_range2'] = df['restaurant_id'].apply(inspect)
for row in df[df.price_range.isna()].iloc:
    row.price_range = grouped[grouped.restaurant_id == row.restaurant_id].price_range.values[0]
    df.iloc[row.name] = row.values

df.price_range.value_counts()

C:\Users\Aliboyev Abror\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


$$ - $$$    29647
$            8768
$$$$         1565
Name: price_range, dtype: int64

In [63]:
price_range = pd.DataFrame(np.concatenate((np.array([df.id_ta.values]).T, np.zeros((len(df), 3), dtype=int)), axis=1), columns=['id_ta', 'price_low','price_average','price_high'])
price_range['price_low'] = df['price_range'].apply(lambda x: 1 if x == '$' else 0)
price_range['price_average'] = df['price_range'].apply(lambda x: 1 if x == '$$ - $$$' else 0)
price_range['price_high'] = df['price_range'].apply(lambda x: 1 if x == '$$$$' else 0)

price_range.head()

,id_ta,price_low,price_average,price_high
0,d1912643,0,1,0
1,d7992032,0,1,0
2,d8632781,0,0,1
3,d1358776,0,1,0
4,d6864963,0,1,0


### number_of_reviews

In [64]:
# Заполняем nan значении с mean
avg_n_reviews = df.number_of_reviews.mean()
df.number_of_reviews.fillna(int(avg_n_reviews), inplace=True)

### reviews

In [65]:
date_ptn = re.compile('\'\d+/\d+/\d+\'?')
df['review_dates'] = df['reviews'].apply(lambda x: [dt.strptime(item, "'%m/%d/%Y'") for item in date_ptn.findall(x)])
df['review_dd'] = df['review_dates'].apply(lambda x: (x[0] - x[1]).days if len(x) >1 else 0)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39980 entries, 0 to 39979
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      39980 non-null  object 
 1   city               39980 non-null  object 
 2   cuisine_style      30701 non-null  object 
 3   ranking            39980 non-null  float64
 4   rating             39980 non-null  float64
 5   price_range        39980 non-null  object 
 6   number_of_reviews  39980 non-null  float64
 7   reviews            39980 non-null  object 
 8   url_ta             39980 non-null  object 
 9   id_ta              39980 non-null  object 
 10  cuisines_count     39980 non-null  int64  
 11  review_dates       39980 non-null  object 
 12  review_dd          39980 non-null  int64  
dtypes: float64(3), int64(2), object(8)
memory usage: 4.0+ MB


In [66]:
# Прежде чем вносить опасные изменнения в наш датафрейм, сохраним его
df.to_csv('modified.csv')


In [67]:
df = df.merge(price_range, on='id_ta')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39980 entries, 0 to 39979
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      39980 non-null  object 
 1   city               39980 non-null  object 
 2   cuisine_style      30701 non-null  object 
 3   ranking            39980 non-null  float64
 4   rating             39980 non-null  float64
 5   price_range        39980 non-null  object 
 6   number_of_reviews  39980 non-null  float64
 7   reviews            39980 non-null  object 
 8   url_ta             39980 non-null  object 
 9   id_ta              39980 non-null  object 
 10  cuisines_count     39980 non-null  int64  
 11  review_dates       39980 non-null  object 
 12  review_dd          39980 non-null  int64  
 13  price_low          39980 non-null  int64  
 14  price_average      39980 non-null  int64  
 15  price_high         39980 non-null  int64  
dtypes: float64(3), int64(5

In [68]:
# переведём в нижний регистр названия столбцов
cuisines.columns = cuisines.columns.str.lower()
# заменим пробелы в названиях столбцов на символ нижний подчерк
cuisines.columns = cuisines.columns.str.replace(' ', '_')
df = df.merge(cuisines, on='id_ta')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39980 entries, 0 to 39979
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   restaurant_id                39980 non-null  object 
 1   city                         39980 non-null  object 
 2   cuisine_style                30701 non-null  object 
 3   ranking                      39980 non-null  float64
 4   rating                       39980 non-null  float64
 5   price_range                  39980 non-null  object 
 6   number_of_reviews            39980 non-null  float64
 7   reviews                      39980 non-null  object 
 8   url_ta                       39980 non-null  object 
 9   id_ta                        39980 non-null  object 
 10  cuisines_count               39980 non-null  int64  
 11  review_dates                 39980 non-null  object 
 12  review_dd                    39980 non-null  int64  
 13  price_low       

### ratings

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39980 entries, 0 to 39979
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   restaurant_id                39980 non-null  object 
 1   city                         39980 non-null  object 
 2   cuisine_style                30701 non-null  object 
 3   ranking                      39980 non-null  float64
 4   rating                       39980 non-null  float64
 5   price_range                  39980 non-null  object 
 6   number_of_reviews            39980 non-null  float64
 7   reviews                      39980 non-null  object 
 8   url_ta                       39980 non-null  object 
 9   id_ta                        39980 non-null  object 
 10  cuisines_count               39980 non-null  int64  
 11  review_dates                 39980 non-null  object 
 12  review_dd                    39980 non-null  int64  
 13  price_low       

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [71]:
# объеденяем датафреймы

# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['restaurant_id', 'city', 'cuisine_style', 'rating',
       'price_range',  'reviews', 'url_ta', 'id_ta',
        'review_dates'], axis = 1)
y = df['rating']
X

,ranking,number_of_reviews,cuisines_count,review_dd,price_low,price_average,price_high,cuisine_vietnamese,cuisine_contemporary,cuisine_barbecue,...,cuisine_pizza,cuisine_asian,cuisine_french,cuisine_bar,cuisine_gluten_free_options,cuisine_vegan_options,cuisine_italian,cuisine_mediterranean,cuisine_european,cuisine_vegetarian_friendly
0,5570.0,194.0,3,41,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,1537.0,10.0,1,382,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,353.0,688.0,7,2,0,0,1,0,0,0,...,0,1,0,0,1,1,0,0,0,1
3,3458.0,3.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,621.0,84.0,3,272,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39975,500.0,79.0,4,34,0,1,0,0,0,0,...,0,0,0,0,1,1,1,0,0,1
39976,6341.0,542.0,5,9,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,1,1
39977,1652.0,4.0,2,3127,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39978,641.0,70.0,5,23,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [72]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [73]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [74]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [75]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [82]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
# MAE: 0.299759122418352


MAE: 0.3584772088425165
